In [1]:
import pandas as pd

In [ ]:
features = pd.read_csv('features.csv', index_col = 'Unnamed: 0')

In [ ]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()

for i in ['channel_type','trx_category','Alphabetic Code','cat_mcc']:
    le.fit(features_y[i].unique())
    features_y[i] = le.transform(features_y[i])

In [ ]:
features['timestamp'] = pd.Series(features.datetime.values.astype(np.int64) // 10 ** 9)

In [ ]:
from bs4 import BeautifulSoup
import urllib2

In [ ]:
hdr={'User-Agent':'Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US; rv:1.9.0.7) Gecko/2009021910 Firefox/3.0.7',} 
req = urllib2.Request('http://mcc-codes.ru/code', headers=hdr)
response = urllib2.urlopen(req)
the_page = response.read()

In [ ]:
p = []
d = []
tb=[]
for tables in soup.find('table', class_ = 'table table-hover table-bordered table-sm'):
    tb.append(tables)
for tb1 in tb[3].find_all('tr'):
    d.append(int(tb1.getText().split(' ')[0].split()[0]))
    p.append(str(tb1.getText().split('\n')[3].encode('utf-8')))

In [ ]:
mccframe = pd.concat([pd.DataFrame(p,columns=['cat_mcc']),
                      pd.DataFrame(d,columns=['MCC']).astype('int64')], 
                     axis=1)

In [ ]:
mcc_groups = features_test.cat_mcc.unique()

def build_features(data):
    aggregated = data.groupby('cl_id')[[ 
        'channel_type',
    ]].first()
    ids = aggregated.index
    aggregated['cl_id'] = ids
    aggregated['sum_POS'] = data[data.trx_category == 6].groupby('cl_id')['convert-amount-float'].sum()
    for mcc_g in mcc_groups:
        aggregated['sum_%s' % mcc_g] = data[
            (data.cat_mcc == mcc_g)
        ].groupby('cl_id')['convert-amount-float'].sum()
    
    return aggregated.fillna(0)

train_agg = build_features(features_test)

feat = train_agg.columns
print(feat)

In [ ]:
train_agg = train_agg.reset_index(drop=True).drop('channel_type',axis=1)
features_test = pd.merge(features_test, train_agg, on='cl_id', how='left')

In [ ]:
features_exp = features[features['trx_category']==6].groupby('cl_id').apply(lambda x: x.sort_values('timestamp')).reset_index(drop=True)

In [ ]:
def time_func(x):
    list1 = x.timestamp.values[:-1]
    list2 = x.timestamp.values[1:]
    
    features_exp.loc[x.index.values[0], 'expectation'] = 0
    list3 = np.array(map(lambda a,b: a-b, list2, list1))
    
    features_exp.loc[x.index.values[1:], 'expectation'] = list3

In [ ]:
features['month'] = features['PERIOD'].str[3:5]
features['year'] = features['PERIOD'].str[6:10]
features['day'] = features['TRDATETIME'].str[0:2]
features['time'] = features['TRDATETIME'].str[8:10]

In [ ]:
all_days = features[['day','month','year']].drop_duplicates()

In [ ]:
import requests
import json

In [ ]:
frame = pd.DataFrame()
for i in all_days.index.values:
    req = requests.get('http://data.fixer.io/api/'+all_days['year'].loc[i]+
                       '-'+all_days['month'].loc[i]+
                       '-'+all_days['day'].loc[i]+
                       '?access_key=7429e635072542aa60509447acae8dfb'+
                       '&base=EUR'+
                       '&symbols=RUR,EUR,VND,USD,TRY,GEL,AMD,GBP,DOP,THB,CZK,'+
                       'SGD,IDR,CHF,BYR,BGN,AUD,HRK,AED,UAH,PLN,CAD,KZT,ILS,'+
                       'AZN,MAD,ISK,KGS,CNY,SEK,JPY,MXN,QAR,CRC,RUB,HUF,INR,MYR,'+
                       'BRL,MDL,NOK,DKK,HKD,ARS,KRW,TND,MUR,PEN,RSD,LKR,RON,ZAR,'+
                       'SCR,BSD,COP,TWD,PHP,NZD,MVR').json()
    
    if frame.size < 1:
        frame = pd.DataFrame(req['rates'], index=[i])
    else:
        frame = frame.append(pd.DataFrame(req['rates'], index=[i]))

In [ ]:
features['convert-amount'] = 0.0

In [ ]:
for i in all_days.index.values:
    for k in features[(features['day']==all_days.loc[i]['day'])&
                      (features['month']==all_days.loc[i]['month'])&
                      (features['year']==all_days.loc[i]['year'])].index.values:
        
        try:
            features.set_value(k, 'convert-amount', frame.loc[i][features.loc[k]['Alphabetic Code']])
        except KeyError:
            features.set_value(k, 'convert-amount', 0)
        except ValueError:
            features.set_value(k, 'convert-amount', 0)